In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import backtrader as bt
from backtrader.sizers import PercentSizer
import backtrader.analyzers as btanalyzers

from termcolor import colored

%matplotlib inline
%matplotlib widget

In [12]:
APPLE_FILE_CSV = r'../data/stooq/data/daily/us/nasdaq stocks/1/aapl.csv'

In [13]:
def print_trade_analysis(analyzer):
    # Get the results we are interested in
    if not analyzer.get("total"):
        return

    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total, 2)
    strike_rate = round((total_won / total_closed) * 2)

    # Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate', 'Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed, total_won, total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]

    # Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)

    # Print the rows
    print_list = [h1, r1, h2, r2]
    row_format = "{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format('', *row))


def print_sqn(analyzer):
    sqn = round(analyzer.sqn, 2)
    print('SQN: {}'.format(sqn))

In [14]:
class FullMoney(PercentSizer):
    params = (
        ('percents', 99),
    )

In [15]:
### See: https://github.com/rodrigo-brito/backtrader-binance-bot

class StrategyBase(bt.Strategy):
    def __init__(self):
        self.order = None
        self.last_operation = "SELL"
        self.status = "DISCONNECTED"
        self.bar_executed = 0
        self.buy_price_close = None
        self.soft_sell = False
        self.hard_sell = False
        self.log("Base strategy initialized")

    def reset_sell_indicators(self):
        self.soft_sell = False
        self.hard_sell = False
        self.buy_price_close = None

    def short(self):
        if self.last_operation == "SELL":
            return

        self.log("Sell ordered: $%.2f" % self.data0.close[0])
        return self.sell()

    def long(self):
        if self.last_operation == "BUY":
            return

        self.log("Buy ordered: $%.2f" % self.data0.close[0], True)
        self.buy_price_close = self.data0.close[0]
        price = self.data0.close[0]

        return self.buy()

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
#             self.log('ORDER ACCEPTED/SUBMITTED')
            self.order = order
            return

        if order.status in [order.Expired]:
            self.log('BUY EXPIRED', True)

        elif order.status in [order.Completed]:
            if order.isbuy():
                self.last_operation = "BUY"
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm), True)

            else:  # Sell
                self.last_operation = "SELL"
                self.reset_sell_indicators()
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm), True)

            # Sentinel to None: new orders allowed
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected: Status %s - %s' % (order.Status[order.status],
                                                                         self.last_operation), True)

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        color = 'green'
        if trade.pnl < 0:
            color = 'red'

        self.log(colored('OPERATION PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm), color), True)

    def log(self, txt, send_telegram=False, color=None):

        value = datetime.now()
        if len(self) > 0:
            value = self.data0.datetime.datetime()

        if color:
            txt = colored(txt, color)

        print('[%s] %s' % (value.strftime("%d-%m-%y %H:%M"), txt))

In [16]:
class StochRSI(bt.Indicator):
    lines = ('fastk', 'fastd',)

    params = (
        ('k_period', 3),
        ('d_period', 3),
        ('period', 14),
        ('stoch_period', 14),
        ('upperband', 70.0),
        ('lowerband', 30.0),
    )

    def __init__(self, base_indicator):
        rsi_ll = bt.ind.Lowest(base_indicator, period=self.p.period)
        rsi_hh = bt.ind.Highest(base_indicator, period=self.p.period)
        stochrsi = (base_indicator - rsi_ll) / ((rsi_hh - rsi_ll) + 0.00001)

        self.l.fastk = k = bt.indicators.MovingAverageSimple(100.0 * stochrsi, period=self.p.k_period)
        self.l.fastd = bt.indicators.MovingAverageSimple(k, period=self.p.d_period)

In [17]:
class BasicRSI(StrategyBase):
    params = dict(
        period_ema_fast=10,
        period_ema_slow=100
    )

    def __init__(self):
        StrategyBase.__init__(self)
        self.log("Using RSI/EMA strategy")

        self.ema_fast = bt.indicators.EMA(period=self.p.period_ema_fast)
        self.ema_slow = bt.indicators.EMA(period=self.p.period_ema_slow)
        self.rsi = bt.indicators.RelativeStrengthIndex()

        self.profit = 0

    def update_indicators(self):
        self.profit = 0
        if self.buy_price_close and self.buy_price_close > 0:
            self.profit = float(self.data0.close[0] - self.buy_price_close) / self.buy_price_close

    def next(self):
        self.update_indicators()

        if self.order:  # waiting for pending order
            return

        # stop Loss
        if self.profit < -0.03:
            self.log("STOP LOSS: percentage %.3f %%" % self.profit)
            self.short()

        if self.last_operation != "BUY":
            if self.rsi < 30 and self.ema_fast > self.ema_slow:
                self.long()

        if self.last_operation != "SELL":
            if self.rsi > 70:
                self.short()

In [18]:
class SimpleRSI(StrategyBase):
    def __init__(self):
        StrategyBase.__init__(self)
        self.log("Using RSI strategy")

        self.rsi = bt.indicators.RelativeStrengthIndex()

        self.profit = 0

    def update_indicators(self):
        self.profit = 0
        if self.buy_price_close and self.buy_price_close > 0:
            self.profit = float(self.data0.close[0] - self.buy_price_close) / self.buy_price_close

    def next(self):
        self.update_indicators()

        if self.order:  # waiting for pending order
            return

        if self.last_operation != "BUY":
            if self.rsi < 30:
                self.long()

        if self.last_operation != "SELL":
            if self.rsi > 70:
                self.short()

In [29]:
#Get Data
data1 = bt.feeds.GenericCSVData(dataname=APPLE_FILE_CSV,params = (
        ('nullvalue', float('NaN')),
        ('dtformat', '%Y%m%d'),
        ('timeframe', bt.TimeFrame.Days),
        ('fromdate', datetime(2000, 1, 1)),
        ('todate', datetime(2000, 12, 31)),
        ('compression', 1),
        ('datetime', 0),
        ('time', -1),
        ('open', 1),
        ('high', 2),
        ('low', 3,
        ('close', 4),
        ('volume', 5)
        )))

cerebro = bt.Cerebro(quicknotify=True)

cerebro.adddata(data1)

broker = cerebro.getbroker()
broker.setcommission(commission=0.001)  # Simulating exchange fee
broker.setcash(100000.0)
cerebro.addsizer(FullMoney)

# Analyzers to evaluate trades and strategies
# SQN = Average( profit / risk ) / StdDev( profit / risk ) x SquareRoot( number of trades )
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Years)
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharperatio')
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annual')
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(btanalyzers.DrawDown)

# Include Strategy
cerebro.addstrategy(SimpleRSI)

# Starting backtrader bot
initial_value = cerebro.broker.getvalue()
print('Starting Portfolio Value: %.2f' % initial_value)
results = cerebro.run()
strat = results[0]

portfolio_stats = strat.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)

# Print analyzers - results
final_value = cerebro.broker.getvalue()
print('Final Portfolio Value: %.2f' % final_value)
print('Profit %.3f%%' % ((final_value - initial_value) / initial_value * 100))
print_trade_analysis(results[0].analyzers.ta.get_analysis())
print_sqn(results[0].analyzers.sqn.get_analysis())

# If no name has been specified, the name is the class name lowercased
tret_analyzer = strat.analyzers.getbyname('timereturn')
maxdrawdown = strat.analyzers.drawdown.get_analysis()['max']['drawdown']
cagr = strat.analyzers.returns.get_analysis()['rnorm100']
sharpe = strat.analyzers.sharperatio.get_analysis()['sharperatio']
print(f"Max Drawdown: {maxdrawdown:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")

Starting Portfolio Value: 100000.00
[09-04-21 14:27] Base strategy initialized
[09-04-21 14:27] Using RSI strategy
[12-10-84 23:59] Buy ordered: $0.09
[15-10-84 23:59] Order Canceled/Margin/Rejected: Status Margin - SELL
[07-03-85 23:59] Buy ordered: $0.08
[08-03-85 23:59] BUY EXECUTED, Price: 0.08, Cost: 99000.00, Comm 99.00
[16-10-85 23:59] Sell ordered: $0.07
[17-10-85 23:59] SELL EXECUTED, Price: 0.07, Cost: 99000.00, Comm 81.44
[17-10-85 23:59] OPERATION PROFIT, GROSS -17561.13, NET -17741.57
[19-10-87 23:59] Buy ordered: $0.28
[20-10-87 23:59] Order Canceled/Margin/Rejected: Status Margin - SELL
[20-10-87 23:59] Buy ordered: $0.27
[21-10-87 23:59] Order Canceled/Margin/Rejected: Status Margin - SELL
[23-10-87 23:59] Buy ordered: $0.27
[26-10-87 23:59] BUY EXECUTED, Price: 0.27, Cost: 79053.74, Comm 79.05
[03-03-88 23:59] Sell ordered: $0.36
[04-03-88 23:59] SELL EXECUTED, Price: 0.35, Cost: 79053.74, Comm 105.47
[04-03-88 23:59] OPERATION PROFIT, GROSS 26420.81, NET 26236.28
[11-

In [28]:
import matplotlib
matplotlib.use('agg')

%matplotlib inline
%matplotlib widget

cerebro.plot(iplot= False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[<Figure size 432x288 with 5 Axes>]]